In [44]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [45]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from plotly.offline import init_notebook_mode, iplot
import chart_studio.plotly as py
import plotly.graph_objs as go

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [46]:
train = pd.read_csv('train.csv', parse_dates=['date'])

In [47]:
test = pd.read_csv('test.csv', parse_dates=['date'])

In [48]:
test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [49]:
lag_size = (test['date'].max().date() - train['date'].max().date()).days
print('Forecast Lag Size', lag_size)

Forecast Lag Size 90


In [50]:
lag_size = 30

In [51]:
train = train[(train['date'] >= '2017-01-01')]

In [63]:
train_gp = train.sort_values('date').groupby(['item', 'store', 'date'], as_index=False)
train_gp = train_gp.agg({'sales':['mean']})
train_gp.columns = ['item', 'store', 'date', 'sales']
train_gp.tail(35)

,item,store,date,sales
182465,50,10,2017-11-27,75
182466,50,10,2017-11-28,68
182467,50,10,2017-11-29,76
182468,50,10,2017-11-30,73
182469,50,10,2017-12-01,66
182470,50,10,2017-12-02,49
182471,50,10,2017-12-03,75
182472,50,10,2017-12-04,54
182473,50,10,2017-12-05,68
182474,50,10,2017-12-06,60


In [53]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [54]:
window = 29
lag = lag_size
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)
series.head()

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),...,sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+30),store(t+30),sales(t+30)
29,1.0,1.0,19.0,1.0,1.0,15.0,1.0,1.0,10.0,1.0,...,16.0,1.0,1.0,24.0,1,1,9,1.0,1.0,16.0
30,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,...,24.0,1.0,1.0,9.0,1,1,17,1.0,1.0,23.0
31,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,...,9.0,1.0,1.0,17.0,1,1,15,1.0,1.0,22.0
32,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,...,17.0,1.0,1.0,15.0,1,1,17,1.0,1.0,13.0
33,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,...,15.0,1.0,1.0,17.0,1,1,24,1.0,1.0,25.0


In [55]:
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]

In [56]:
columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item', 'store']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)

In [62]:
series

,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t),sales(t+30)
29,19.0,15.0,10.0,16.0,14.0,24.0,14.0,20.0,18.0,11.0,...,21.0,9.0,18.0,14.0,13.0,10.0,16.0,24.0,9,16.0
30,15.0,10.0,16.0,14.0,24.0,14.0,20.0,18.0,11.0,14.0,...,9.0,18.0,14.0,13.0,10.0,16.0,24.0,9.0,17,23.0
31,10.0,16.0,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,...,18.0,14.0,13.0,10.0,16.0,24.0,9.0,17.0,15,22.0
32,16.0,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,7.0,...,14.0,13.0,10.0,16.0,24.0,9.0,17.0,15.0,17,13.0
33,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,7.0,16.0,...,13.0,10.0,16.0,24.0,9.0,17.0,15.0,17.0,24,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182465,89.0,69.0,80.0,81.0,79.0,76.0,111.0,108.0,66.0,86.0,...,88.0,62.0,81.0,81.0,87.0,81.0,78.0,113.0,75,63.0
182466,69.0,80.0,81.0,79.0,76.0,111.0,108.0,66.0,86.0,79.0,...,62.0,81.0,81.0,87.0,81.0,78.0,113.0,75.0,68,59.0
182467,80.0,81.0,79.0,76.0,111.0,108.0,66.0,86.0,79.0,89.0,...,81.0,81.0,87.0,81.0,78.0,113.0,75.0,68.0,76,74.0
182468,81.0,79.0,76.0,111.0,108.0,66.0,86.0,79.0,89.0,85.0,...,81.0,87.0,81.0,78.0,113.0,75.0,68.0,76.0,73,62.0


In [58]:
X_valid

,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
117349,103.0,102.0,120.0,129.0,122.0,126.0,148.0,112.0,93.0,101.0,...,159.0,112.0,98.0,102.0,117.0,144.0,139.0,154.0,103.0,115
4370,97.0,95.0,87.0,109.0,68.0,74.0,79.0,94.0,83.0,65.0,...,60.0,77.0,72.0,68.0,56.0,41.0,52.0,45.0,60.0,73
45664,47.0,37.0,60.0,55.0,53.0,49.0,43.0,51.0,41.0,53.0,...,49.0,56.0,53.0,55.0,57.0,66.0,43.0,45.0,44.0,43
24414,42.0,41.0,38.0,42.0,51.0,54.0,51.0,41.0,41.0,49.0,...,52.0,36.0,43.0,36.0,47.0,51.0,58.0,55.0,36.0,39
36702,86.0,94.0,88.0,47.0,71.0,79.0,75.0,93.0,101.0,124.0,...,92.0,90.0,104.0,102.0,87.0,82.0,83.0,102.0,104.0,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99419,137.0,126.0,163.0,82.0,110.0,115.0,115.0,135.0,111.0,136.0,...,117.0,130.0,152.0,155.0,100.0,123.0,110.0,124.0,129.0,127
162285,105.0,75.0,73.0,87.0,92.0,80.0,89.0,100.0,69.0,62.0,...,75.0,82.0,65.0,77.0,84.0,73.0,78.0,96.0,89.0,71
22561,78.0,86.0,76.0,79.0,91.0,89.0,116.0,58.0,69.0,81.0,...,103.0,74.0,78.0,82.0,62.0,87.0,101.0,95.0,76.0,80
39174,99.0,106.0,82.0,97.0,114.0,89.0,104.0,104.0,126.0,76.0,...,98.0,113.0,121.0,80.0,92.0,88.0,92.0,100.0,127.0,107


In [14]:
# Label
labels_col = 'sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (100782, 30)
Validation set shape (67188, 30)


,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),...,sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
36102,86.0,87.0,93.0,93.0,107.0,108.0,78.0,62.0,81.0,100.0,...,67.0,79.0,99.0,101.0,91.0,108.0,109.0,64.0,91.0,84
40969,81.0,71.0,84.0,102.0,61.0,69.0,76.0,66.0,86.0,84.0,...,74.0,67.0,85.0,87.0,101.0,62.0,78.0,65.0,84.0,100
110679,70.0,67.0,38.0,48.0,62.0,70.0,51.0,67.0,64.0,45.0,...,65.0,63.0,87.0,44.0,64.0,60.0,61.0,55.0,80.0,70
175614,30.0,45.0,21.0,23.0,25.0,31.0,37.0,32.0,43.0,23.0,...,45.0,45.0,34.0,22.0,42.0,38.0,25.0,36.0,36.0,39
136633,69.0,83.0,71.0,94.0,94.0,62.0,61.0,57.0,68.0,84.0,...,70.0,66.0,69.0,68.0,84.0,98.0,66.0,64.0,66.0,82


In [42]:
Y_train

array([ 76., 103.,  77., ...,  65., 105., 108.])

In [19]:
epochs = 40
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

In [20]:

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               3968      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 168,833
Trainable params: 168,833
Non-trainable params: 0
_________________________________________________________________


In [21]:
NN_model1 = NN_model.fit(X_train.values, Y_train, validation_data=(X_valid.values, Y_valid), epochs=epochs, verbose=2)

Epoch 1/40
3150/3150 - 11s - loss: 200.6324 - mean_squared_error: 200.6324 - val_loss: 208.5698 - val_mean_squared_error: 208.5698
Epoch 2/40
3150/3150 - 11s - loss: 183.3342 - mean_squared_error: 183.3342 - val_loss: 176.1320 - val_mean_squared_error: 176.1320
Epoch 3/40
3150/3150 - 11s - loss: 177.1543 - mean_squared_error: 177.1543 - val_loss: 169.2375 - val_mean_squared_error: 169.2375
Epoch 4/40
3150/3150 - 11s - loss: 174.9010 - mean_squared_error: 174.9010 - val_loss: 171.5947 - val_mean_squared_error: 171.5947
Epoch 5/40
3150/3150 - 11s - loss: 171.9271 - mean_squared_error: 171.9271 - val_loss: 166.6410 - val_mean_squared_error: 166.6410
Epoch 6/40
3150/3150 - 11s - loss: 170.2517 - mean_squared_error: 170.2517 - val_loss: 175.9616 - val_mean_squared_error: 175.9616
Epoch 7/40
3150/3150 - 11s - loss: 169.3913 - mean_squared_error: 169.3913 - val_loss: 163.9347 - val_mean_squared_error: 163.9347
Epoch 8/40
3150/3150 - 11s - loss: 169.0216 - mean_squared_error: 169.0216 - val_lo

In [29]:
dnn_train_pred = NN_model.predict(X_train.values)
dnn_valid_pred = NN_model.predict(X_valid.values)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, dnn_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, dnn_valid_pred)))

Train rmse: 12.667693571336839
Validation rmse: 12.714385698041262


In [30]:
def mean_absolute_percentage_error(y_true, y_pred): 

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [31]:
Y_valid = Y_valid.ravel().tolist()

In [32]:
type(Y_valid)

list

In [33]:
Y = np.array(Y_valid)
type(Y)

numpy.ndarray

In [37]:
dnn_valid_pred = dnn_valid_pred.ravel().tolist()

In [39]:
dnn = np.array(dnn_valid_pred)
type(dnn)

numpy.ndarray

In [40]:
mean_absolute_percentage_error(Y,dnn)

19.921696597124264

In [61]:
len(dnn_valid_pred)

67188